In [1]:
# Normalización 3D por pasos (Traslación → Escala) y visualización de un frame elegido
# Aplica: centrar en Cadera_D (6) y escalar por hueso (6–8 con fallback 7–9). SIN rotación Kabsch.
# Este preprocesamiento se usa antes de calcular las métricas MPJPE y PCK.

In [2]:
#2
# JSON de entrada: salida de GenerarJSON.ipynb (incluye GT, VIBE y MediaPipe ya con swap de caderas corregido)
# (swap 6↔7 aplicado en GenerarJSON.ipynb)
JSON_IN  = "dataset_3models_fix.json"

# JSON de salida: misma estructura pero con todas las poses normalizadas (centradas + escaladas, sin Kabsch)
JSON_OUT = "dataset_3models_norm_center_scale.ipynb_out.json"

PERSON_ID = 1
MODELS = ["mp", "vibe"]     # ajusta si tus claves son otras
HIP_IDX = 6                 # "Cadera_D"
BONE_RIGHT = (6, 8)         # cadera derecha - rodilla derecha
BONE_LEFT  = (7, 9)         # cadera izquierda - rodilla izquierda (fallback)
SEQ_TO_SHOW = "outdoors_fencing_01"  # solo esta secuencia
FRAME_TO_SHOW = 600         # frame a visualizar en cada paso


In [3]:
#3
# Funciones auxiliares para trabajar con poses 3D:
# - k3d_to_K3 / K3_to_k3d: conversión entre formato (3xK) del JSON y (K,3) en NumPy.
# - find_models_for_person: extrae los modelos (gt, mp, vibe) de una persona dada.
# - valid_rows / bone_length / choose_bone: selección de filas válidas y cálculo de longitudes de hueso.
# - center_on_joint: centra una pose en torno a una articulación (ej. cadera).
# - kabsch_with_scale: implementación general de Kabsch (NO se usa en la versión final, se deja como referencia).
# - get_frame_index_to_show: elige de forma segura el índice de frame a visualizar.

import json, copy, numpy as np

def k3d_to_K3(mat):
    a = np.asarray(mat, dtype=float)
    if a.ndim != 2 or a.shape[0] < 3: raise ValueError(f"k3d inválido: {a.shape}")
    return a[:3, :].T

def K3_to_k3d(a):
    a = np.asarray(a, dtype=float)
    if a.ndim != 2 or a.shape[1] < 3: raise ValueError(f"K3 inválido: {a.shape}")
    return a[:, :3].T

def find_models_for_person(persons_list, pid):
    cand = None
    for p in persons_list:
        if isinstance(p, dict) and p.get("id") == pid: cand = p; break
    if cand is None and persons_list: cand = persons_list[0]
    return cand.get("models", {}) if isinstance(cand, dict) else {}

def center_on_joint(A, idx):
    if idx >= A.shape[0]: return A, None, False
    j = A[idx]
    if not np.all(np.isfinite(j)): return A, None, False
    return A - j, j, True

def valid_rows(*arrs):
    mask = np.ones(arrs[0].shape[0], dtype=bool)
    for A in arrs: mask &= np.isfinite(A).all(axis=1)
    return mask

def bone_length(A, i, j):
    if max(i,j) >= A.shape[0]: return np.nan
    vi, vj = A[i], A[j]
    if not (np.all(np.isfinite(vi)) and np.all(np.isfinite(vj))): return np.nan
    d = np.linalg.norm(vj - vi)
    return d if np.isfinite(d) else np.nan

def choose_bone(G, P, right=(6,8), left=(7,9)):
    lg = bone_length(G, *right); lp = bone_length(P, *right)
    if np.isfinite(lg) and lg>0 and np.isfinite(lp) and lp>0: return right, lg, lp
    lg = bone_length(G, *left);  lp = bone_length(P, *left)
    if np.isfinite(lg) and lg>0 and np.isfinite(lp) and lp>0: return left, lg, lp
    return None, np.nan, np.nan

def kabsch_with_scale(Pc, Gc):
    H = Pc.T @ Gc
    U, S, Vt = np.linalg.svd(H, full_matrices=False)
    R = Vt.T @ U.T
    if np.linalg.det(R) < 0:
        Vt[-1,:] *= -1
        R = Vt.T @ U.T
    PR = Pc @ R
    num = np.trace(PR.T @ Gc); den = np.trace(Pc.T @ Pc)
    s = float(num/den) if den>0 else 1.0
    Pal = PR * s
    return Pal, R, s

def get_frame_index_to_show(frames, desired_index):
    if not frames: return None, None
    idx = desired_index if desired_index < len(frames) else len(frames)-1
    return idx, frames[idx]


In [4]:
#4
# Cargar el JSON de entrada y listar las secuencias disponibles (video_id / sequence_id)
with open(JSON_IN, "r") as f:
    data = json.load(f)

sequences = data.get("sequences", [])
if isinstance(sequences, dict): sequences = list(sequences.values())

print(f"Secuencias detectadas: {len(sequences)}")
names = [(s.get("video_id") or s.get("sequence_id") or f"seq_{i:03d}") for i,s in enumerate(sequences)]
print("\\n".join(names)) # sirve para comprobar que están las secuencias esperadas


Secuencias detectadas: 20
courtyard_backpack_00\ncourtyard_bodyScannerMotions_00\ncourtyard_box_00\ncourtyard_jumpBench_01\ncourtyard_laceShoe_00\ncourtyard_relaxOnBench_00\ncourtyard_relaxOnBench_01\nflat_guitar_01\noutdoors_climbing_00\noutdoors_climbing_01\noutdoors_climbing_02\noutdoors_crosscountry_00\noutdoors_fencing_01\noutdoors_freestyle_00\noutdoors_freestyle_01\noutdoors_golf_00\noutdoors_parcours_00\noutdoors_parcours_01\noutdoors_slalom_00\noutdoors_slalom_01


In [5]:
#5
# Utilidades de visualización (Matplotlib):
# dibujar el esqueleto de 14 articulaciones para GT, VIBE y MediaPipe en un mismo frame.
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401

KEYPOINT_NAMES = [
    "Hombro_D","Hombro_I","Codo_D","Codo_I","Muñeca_D","Muñeca_I",
    "Cadera_D","Cadera_I","Rodilla_D","Rodilla_I","Tobillo_D","Tobillo_I",
    "Cuello","Cabeza"
]
CONNECTIONS_14 = [(0,2),(2,4),(1,3),(3,5),(6,8),(8,10),(7,9),(9,11),(6,7),(6,0),(7,1),(0,12),(1,12),(12,13)]

# Dibuja un esqueleto 3D (14 joints) con un color y marcador específicos (gt, mp o vibe).
def _plot_skeleton(ax, K3, color, label=None, marker="o", lw=1.5):
    ax.plot(K3[:,0], K3[:,1], K3[:,2], linestyle="None", marker=marker, label=label, color=color)
    for i, j in CONNECTIONS_14:
        if i < len(K3) and j < len(K3):
            if (not np.any(np.isnan(K3[i]))) and (not np.any(np.isnan(K3[j]))):
                ax.plot([K3[i,0], K3[j,0]],
                        [K3[i,1], K3[j,1]],
                        [K3[i,2], K3[j,2]], color=color, linewidth=lw)

def _set_equal_axes(ax, pts_list, pad=0.05):
    pts = [p for p in pts_list if p is not None]
    if not pts: return
    P = np.concatenate(pts, axis=0)
    P = P[np.isfinite(P).all(axis=1)]
    if P.size == 0: return
    mins, maxs = np.min(P, axis=0), np.max(P, axis=0)
    center = (mins + maxs) / 2.0
    r = max(maxs - mins) / 2.0
    r *= (1.0 + pad)
    ax.set_xlim(center[0]-r, center[0]+r)
    ax.set_ylim(center[1]-r, center[1]+r)
    ax.set_zlim(center[2]-r, center[2]+r)
    try: ax.set_box_aspect((1,1,1))
    except Exception: pass  # versiones antiguas de mpl

def plot_frame_K3_overlaid(seq_name, K3_gt, K3_mp=None, K3_vibe=None, title_suffix=""):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    if K3_gt is not None:   _plot_skeleton(ax, K3_gt,  color="green", label="gt",   marker="o")
    if K3_mp is not None:   _plot_skeleton(ax, K3_mp,  color="blue",  label="mp",   marker="x")
    if K3_vibe is not None: _plot_skeleton(ax, K3_vibe,color="red",   label="vibe", marker="^")

    _set_equal_axes(ax, [K3_gt, K3_mp, K3_vibe])
    ax.set_title(f"{seq_name} {title_suffix}")
    ax.set_xlabel("X"); ax.set_ylabel("Y"); ax.set_zlabel("Z")
    ax.legend(); plt.tight_layout(); plt.show()


In [6]:
#6
# Paso 1 — Corrección de traslación (centrado en Cadera_D=6) y vista interactiva SOLO de una secuencia
# Resultado: proc_centered → mismas poses que el JSON original pero todas centradas en la cadera (joint 6).
try:
    SEQ_TO_SHOW
except NameError:
    SEQ_TO_SHOW = "outdoors_fencing_01"

proc_centered = []
for s in sequences:
    name = s.get("video_id") or s.get("sequence_id") or "unknown_seq"
    frames = s.get("frames", [])
    seq_center = {"name": name, "frames": []}
    for fr in frames:
        models = find_models_for_person(fr.get("persons", []), PERSON_ID)
        if "gt" not in models or "k3d" not in models["gt"]:
            seq_center["frames"].append(None); continue
        try: GT_K3 = k3d_to_K3(models["gt"]["k3d"])
        except: seq_center["frames"].append(None); continue

        pack = {"gt": None, "mp": None, "vibe": None}
        try:
            Gc, _, okg = center_on_joint(GT_K3, HIP_IDX)
            pack["gt"] = Gc if okg else None
        except: pass

        for mk in MODELS:
            if mk in models and "k3d" in models[mk]:
                try:
                    PR_K3 = k3d_to_K3(models[mk]["k3d"])
                    Pc, _, okp = center_on_joint(PR_K3, HIP_IDX)
                    pack[mk] = Pc if okp else None
                except: pass
        seq_center["frames"].append(pack)
    proc_centered.append(seq_center)

# ---------- Plotly interactivo ----------
import plotly.graph_objects as go
import numpy as np

# Fallback de conexiones si tu JSON no trae 'connections'
CONNECTIONS_14 = [(0,2),(2,4),(1,3),(3,5),(6,8),(8,10),(7,9),(9,11),(6,7),(6,12),(7,12),(12,13)]
CONNS_GTVIBE = (data.get("connections", {}) or {}).get("gt_vibe", CONNECTIONS_14)
CONNS_MP     = (data.get("connections", {}) or {}).get("mediapipe", CONNECTIONS_14)
COLORS = {"gt":"green","mp":"blue","vibe":"red"}

def plot_frame3d_plotly(seq_name, K3_gt, K3_mp=None, K3_vibe=None, title_suffix=""):
    fig = go.Figure()
    stack = []
    def add_model(pts, model_key, label):
        if pts is None: return
        col = COLORS[model_key]
        stack.append(pts)
        conns = CONNS_MP if model_key=="mp" else CONNS_GTVIBE
        # puntos
        fig.add_trace(go.Scatter3d(x=pts[:,0], y=pts[:,1], z=pts[:,2],
                                   mode="markers", marker=dict(size=5, color=col),
                                   name=label))
        # líneas
        seg_x, seg_y, seg_z = [], [], []
        K = len(pts)
        for a,b in conns:
            if 0<=a<K and 0<=b<K and np.all(np.isfinite(pts[[a,b]])):
                seg_x += [pts[a,0], pts[b,0], None]
                seg_y += [pts[a,1], pts[b,1], None]
                seg_z += [pts[a,2], pts[b,2], None]
        fig.add_trace(go.Scatter3d(x=seg_x, y=seg_y, z=seg_z, mode="lines",
                                   line=dict(width=3, color=col),
                                   name=f"{label} bones", showlegend=False))
    add_model(K3_gt,   "gt",   "GT")
    add_model(K3_mp,   "mp",   "MP")
    add_model(K3_vibe, "vibe", "VIBE")

    # ejes iguales
    P = np.vstack([p[np.isfinite(p).all(axis=1)] for p in stack if p is not None and p.size])
    mins, maxs = np.min(P, axis=0), np.max(P, axis=0)
    ctr  = (mins+maxs)/2.0; rad = float(np.max((maxs - mins)/2.0)) or 1.0

    fig.update_layout(
        title=f"{seq_name} {title_suffix}",
        scene=dict(
            xaxis=dict(range=[ctr[0]-rad, ctr[0]+rad], title="X"),
            yaxis=dict(range=[ctr[1]-rad, ctr[1]+rad], title="Y"),
            zaxis=dict(range=[ctr[2]-rad, ctr[2]+rad], title="Z"),
            aspectmode="cube",
        ), width=800, height=700, showlegend=True
    )
    fig.show()

# Visualizar SOLO la secuencia pedida (GT, MP y VIBE centrados en la cadera en un frame ejemplo)
seq_center = next((sc for sc in proc_centered if sc["name"] == SEQ_TO_SHOW), None)
if seq_center:
    idx, fr_pack = get_frame_index_to_show(seq_center["frames"], FRAME_TO_SHOW)
    if fr_pack:
        plot_frame3d_plotly(seq_center["name"], fr_pack.get("gt"), fr_pack.get("mp"), fr_pack.get("vibe"),
                            title_suffix="— centrado")
else:
    print("Secuencia no encontrada:", SEQ_TO_SHOW)



In [7]:
#7
# Paso 2 — Normalización por escalado (hueso 6–8; fallback 7–9) con vista interactiva SOLO de una secuencia
# Resultado: proc_scaled → GT sin cambios y modelos escalados para igualar la longitud del hueso de referencia.
try:
    SEQ_TO_SHOW
except NameError:
    SEQ_TO_SHOW = "outdoors_fencing_01"

proc_scaled = []
for seq_center in proc_centered:
    name = seq_center["name"]; frames_c = seq_center["frames"]
    seq_sc = {"name": name, "frames": []}
    for fr_pack in frames_c:
        if not fr_pack or fr_pack.get("gt") is None:
            seq_sc["frames"].append(None); continue
        Gc = fr_pack["gt"]
        pack = {"gt": Gc, "mp": None, "vibe": None}
        for mk in MODELS:
            Pc = fr_pack.get(mk, None)
            if Pc is None: continue
            pair, lg, lp = choose_bone(Gc, Pc, right=BONE_RIGHT, left=BONE_LEFT)
            if pair is None: pack[mk] = Pc
            else:
                pre_s = lg / lp if lp > 0 else 1.0
                pack[mk] = Pc * pre_s
        seq_sc["frames"].append(pack)
    proc_scaled.append(seq_sc)

# Vista de GT, MP y VIBE ya centrados + escalados para la secuencia de ejemplo
# Reutiliza la función Plotly de la celda anterior
seq_sc = next((sc for sc in proc_scaled if sc["name"] == SEQ_TO_SHOW), None)
if seq_sc:
    idx, fr_pack = get_frame_index_to_show(seq_sc["frames"], FRAME_TO_SHOW)
    if fr_pack:
        plot_frame3d_plotly(seq_sc["name"], fr_pack.get("gt"), fr_pack.get("mp"), fr_pack.get("vibe"),
                            title_suffix="— centrado + escalado")
else:
    print("Secuencia no encontrada:", SEQ_TO_SHOW)


In [8]:
#8
# Paso 3 — (sin Kabsch) Solo reutilizar las poses centradas + escaladas
# A partir de aquí trabajamos directamente con 'proc_scaled' y lo renombramos como 'proc_aligned'
# para no cambiar el resto del pipeline (guardado del JSON y cálculo de métricas).

proc_aligned = proc_scaled

# Vista interactiva opcional sobre la misma secuencia
try:
    SEQ_TO_SHOW
except NameError:
    SEQ_TO_SHOW = "outdoors_fencing_01"

seq_al = next((sc for sc in proc_aligned if sc["name"] == SEQ_TO_SHOW), None)
if seq_al:
    idx, fr_pack = get_frame_index_to_show(seq_al["frames"], FRAME_TO_SHOW)
    if fr_pack:
        plot_frame3d_plotly(
            seq_al["name"],
            fr_pack.get("gt"),
            fr_pack.get("mp"),
            fr_pack.get("vibe"),
            title_suffix="— centrado + escalado (sin Kabsch)"
        )
else:
    print("Secuencia no encontrada:", SEQ_TO_SHOW)



In [9]:
# Pausa
#raise SystemExit


In [10]:
#9
# Guardar JSON final (conversión a listas para evitar ndarrays en el dump)
import copy, json

out_data = copy.deepcopy(data)
seqs_out = out_data.get("sequences", [])

def _as_list_3xK(K3):
    return K3_to_k3d(K3).tolist()  # (K,3)->(3,K) y a listas nativas

if isinstance(seqs_out, dict):
    name_to_seq = {}
    for k, v in out_data["sequences"].items():
        nm = v.get("video_id") or v.get("sequence_id") or str(k)
        name_to_seq[nm] = (k, v)
    for seq_al in proc_aligned:
        nm = seq_al["name"]
        if nm not in name_to_seq: continue
        key, seq_ref = name_to_seq[nm]
        frames_ref = seq_ref.get("frames", [])
        for i, fr in enumerate(frames_ref):
            if i >= len(seq_al["frames"]): break
            pack = seq_al["frames"][i]
            if not pack: continue
            models = find_models_for_person(fr.get("persons", []), PERSON_ID)
            for mk in MODELS:
                if mk in models and "k3d" in models[mk] and pack.get(mk) is not None:
                    models[mk]["k3d"] = _as_list_3xK(pack[mk])
else:
    for s_idx, seq_ref in enumerate(seqs_out):
        if s_idx >= len(proc_aligned): break
        seq_al = proc_aligned[s_idx]
        frames_ref = seq_ref.get("frames", [])
        for i, fr in enumerate(frames_ref):
            if i >= len(seq_al["frames"]): break
            pack = seq_al["frames"][i]
            if not pack: continue
            models = find_models_for_person(fr.get("persons", []), PERSON_ID)
            for mk in MODELS:
                if mk in models and "k3d" in models[mk] and pack.get(mk) is not None:
                    models[mk]["k3d"] = _as_list_3xK(pack[mk])

meta = dict(out_data.get("meta", {}))
meta["normalized"] = True
meta["pipeline"] = [
    "center_hip(6)",
    f"bone_scale({BONE_RIGHT[0]}-{BONE_RIGHT[1]} or {BONE_LEFT[0]}-{BONE_LEFT[1]})"
]
out_data["meta"] = meta


with open(JSON_OUT, "w", encoding="utf-8") as f:
    json.dump(out_data, f, ensure_ascii=False)

print("Guardado:", JSON_OUT)


Guardado: dataset_3models_norm_center_scale.ipynb_out.json


In [11]:
#10
from google.colab import files
files.download(JSON_OUT)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
#MPJPE
# === MPJPE por articulación (GT vs VIBE / MP) → CSV por secuencia ===
import json, os, numpy as np, pandas as pd

JSON_PATH = "/content/dataset_3models_norm_center_scale.ipynb_out.json"
OUT_DIR = "/content/mpjpe_csv"
DO_CENTERED = True       # True: además guarda CSVs centrados en la articulación 6
CENTER_JOINT = 6

os.makedirs(OUT_DIR, exist_ok=True)

def to_K3(k3d_3xK):
    X = np.array([np.nan if v is None else float(v) for v in k3d_3xK[0]], dtype=np.float32)
    Y = np.array([np.nan if v is None else float(v) for v in k3d_3xK[1]], dtype=np.float32)
    Z = np.array([np.nan if v is None else float(v) for v in k3d_3xK[2]], dtype=np.float32)
    return np.stack([X, Y, Z], axis=1)  # (K,3)

def seq_to_TK3(seq, model_key, pid=1):
    T = len(seq["frames"])
    sample = seq["frames"][0]["persons"][0]["models"][model_key]["k3d"]
    K = len(sample[0])
    arr = np.full((T, K, 3), np.nan, dtype=np.float32)
    for t, fr in enumerate(seq["frames"]):
        person = next(p for p in fr["persons"] if p["id"] == pid)
        arr[t] = to_K3(person["models"][model_key]["k3d"])
    return arr  # (T,K,3)

def mpjpe_per_joint(P, G, center_joint=None):
    P = P.copy(); G = G.copy()
    if center_joint is not None:
        pj = P[:, center_joint, :]  # (T,3)
        gj = G[:, center_joint, :]
        m = np.isfinite(pj).all(axis=1) & np.isfinite(gj).all(axis=1)
        P[m] = P[m] - pj[m][:, None, :]
        G[m] = G[m] - gj[m][:, None, :]
    D = np.linalg.norm(P - G, axis=2)  # (T,K)
    return np.nanmean(D, axis=0)       # (K,)

with open(JSON_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

kp_names = data["keypoint_names"]
seqs = data["sequences"]

for seq in seqs:
    vid = seq["video_id"]
    GT = seq_to_TK3(seq, "gt")
    MP = seq_to_TK3(seq, "mp")
    VB = seq_to_TK3(seq, "vibe")

    T = min(GT.shape[0], MP.shape[0], VB.shape[0])
    GT, MP, VB = GT[:T], MP[:T], VB[:T]

    # --- Escenario: sin centrar ---
    mp_err = mpjpe_per_joint(MP, GT, center_joint=None)
    vb_err = mpjpe_per_joint(VB, GT, center_joint=None)
    df = pd.DataFrame({
        "MPJPE VIBE": vb_err.astype(np.float32),
        "MPJPE MP":   mp_err.astype(np.float32),
    }, index=kp_names)
    out_csv = os.path.join(OUT_DIR, f"{vid}__original.csv")
    df.to_csv(out_csv)
    print(f"OK: {out_csv}")

    # --- Escenario: centrado en joint 6 (opcional) ---
    if DO_CENTERED:
        mp_err_c = mpjpe_per_joint(MP, GT, center_joint=CENTER_JOINT)
        vb_err_c = mpjpe_per_joint(VB, GT, center_joint=CENTER_JOINT)
        df_c = pd.DataFrame({
            "MPJPE VIBE": vb_err_c.astype(np.float32),
            "MPJPE MP":   mp_err_c.astype(np.float32),
        }, index=kp_names)
        out_csv_c = os.path.join(OUT_DIR, f"{vid}__centered_hip.csv")
        df_c.to_csv(out_csv_c)
        print(f"OK: {out_csv_c}")

print("Listo. CSVs en:", OUT_DIR)

!zip -r /content/mpjpe_csv.zip /content/mpjpe_csv
from google.colab import files; files.download("/content/mpjpe_csv.zip")

OK: /content/mpjpe_csv/courtyard_backpack_00__original.csv
OK: /content/mpjpe_csv/courtyard_backpack_00__centered_hip.csv
OK: /content/mpjpe_csv/courtyard_bodyScannerMotions_00__original.csv
OK: /content/mpjpe_csv/courtyard_bodyScannerMotions_00__centered_hip.csv
OK: /content/mpjpe_csv/courtyard_box_00__original.csv
OK: /content/mpjpe_csv/courtyard_box_00__centered_hip.csv
OK: /content/mpjpe_csv/courtyard_jumpBench_01__original.csv
OK: /content/mpjpe_csv/courtyard_jumpBench_01__centered_hip.csv
OK: /content/mpjpe_csv/courtyard_laceShoe_00__original.csv
OK: /content/mpjpe_csv/courtyard_laceShoe_00__centered_hip.csv
OK: /content/mpjpe_csv/courtyard_relaxOnBench_00__original.csv
OK: /content/mpjpe_csv/courtyard_relaxOnBench_00__centered_hip.csv
OK: /content/mpjpe_csv/courtyard_relaxOnBench_01__original.csv
OK: /content/mpjpe_csv/courtyard_relaxOnBench_01__centered_hip.csv
OK: /content/mpjpe_csv/flat_guitar_01__original.csv
OK: /content/mpjpe_csv/flat_guitar_01__centered_hip.csv
OK: /conte

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
# Lee automáticamente todos los CSV en /content/mpjpe_csv (y subcarpetas),
# los clasifica y arma un .odt con:
# - Todas las tablas por secuencia (originales y centradas)
# - Al final, 2 tablas de PROMEDIO global por articulación (original y centered_hip)

!pip install -q odfpy

import os, glob
import pandas as pd
from google.colab import files
from odf.opendocument import OpenDocumentText
from odf.text import P
from odf.table import Table, TableRow, TableCell, TableColumn
from odf.style import Style, TextProperties

# 1) Buscar CSVs en la carpeta
CSV_DIR = "/content/mpjpe_csv"
paths = sorted(glob.glob(os.path.join(CSV_DIR, "**", "*.csv"), recursive=True))

# 2) Clasificar archivos (promedio / centered / original)
group_centered, group_original, group_promedio = {}, {}, {}

def classify(rel_name: str):
    """
    Clasifica por nombre relativo:
    - 'promedio' si contiene 'promedio', 'mean' o 'global'
    - 'centered' si termina en 'centered_hip.csv' o contiene 'centered'
    - 'original' si termina en 'original.csv' o contiene 'original' o 'raw'
    """
    lname = rel_name.lower()
    if any(k in lname for k in ("promedio_por_articulacion", "promedio", "mean", "global")):
        return "promedio"
    if lname.endswith("centered_hip.csv") or "centered" in lname:
        return "centered"
    if lname.endswith("original.csv") or "original" in lname or "raw" in lname:
        return "original"
    return None

for p in paths:
    rel = os.path.relpath(p, CSV_DIR)  # nombre relativo para que se vea la subcarpeta si aplica
    grp = classify(rel)
    if grp == "promedio":
        group_promedio[rel] = p
    elif grp == "centered":
        group_centered[rel] = p
    elif grp == "original":
        group_original[rel] = p

print("CSV originales encontrados :", len(group_original))
print("CSV centrados encontrados  :", len(group_centered))
print("CSV de promedio ya existentes:", len(group_promedio))

# 3) Crear documento ODT y estilos
doc = OpenDocumentText()

title_style = Style(name="TitleStyle", family="paragraph")
title_style.addElement(TextProperties(attributes={"fontweight": "bold", "fontsize": "16pt"}))
doc.styles.addElement(title_style)

section_style = Style(name="SectionStyle", family="paragraph")
section_style.addElement(TextProperties(attributes={"fontweight": "bold", "fontsize": "18pt", "color": "#1A5276"}))
doc.styles.addElement(section_style)

def add_df_table(doc, df, title):
    """Agregar una tabla al documento a partir de un DataFrame."""
    if title:
        doc.text.addElement(P(stylename=title_style, text=title))

    table = Table(name=title)
    for _ in range(len(df.columns)):
        table.addElement(TableColumn())

    # Encabezado
    header_row = TableRow()
    for col in df.columns:
        cell = TableCell()
        cell.addElement(P(text=str(col)))
        header_row.addElement(cell)
    table.addElement(header_row)

    # Filas
    for _, row in df.iterrows():
        tr = TableRow()
        for val in row.tolist():
            cell = TableCell()
            cell.addElement(P(text=str(val)))
            tr.addElement(cell)
        table.addElement(tr)

    doc.text.addElement(table)
    doc.text.addElement(P(text="\f"))  # salto de página

def read_csv_for_table(path):
    """Leer CSV y dejar, si existe, la columna de articulación con nombre limpio."""
    df = pd.read_csv(path)
    if "Articulación" in df.columns:
        return df
    if "Unnamed: 0" in df.columns:
        df = df.rename(columns={"Unnamed: 0": "Articulación"})
    return df

# 4) Tablas por secuencia (primero centradas, luego originales)

if group_centered:
    doc.text.addElement(P(stylename=section_style, text="Tablas centradas en cadera (centered_hip)"))
    for name in sorted(group_centered.keys()):
        path = group_centered[name]
        df = read_csv_for_table(path)
        add_df_table(doc, df, name)

if group_original:
    doc.text.addElement(P(stylename=section_style, text="Tablas originales (sin centrado adicional)"))
    for name in sorted(group_original.keys()):
        path = group_original[name]
        df = read_csv_for_table(path)
        add_df_table(doc, df, name)

# 5) Calcular PROMEDIOS globales por articulación (solo a partir de los CSV por secuencia)

def promedio_por_articulacion(file_dict):
    """Dado un dict nombre->ruta de CSV (mismo formato), devuelve promedio global por articulación."""
    if not file_dict:
        return None

    dfs = []
    for p in file_dict.values():
        df = pd.read_csv(p)

        # Normalizar nombre de la columna de articulación
        if "Articulación" in df.columns:
            df = df.set_index("Articulación")
        elif "Unnamed: 0" in df.columns:
            df = df.rename(columns={"Unnamed: 0": "Articulación"}).set_index("Articulación")
        else:
            # Si no hay nombre claro, asumir primera columna como nombre de articulación
            df = df.set_index(df.columns[0])

        dfs.append(df)

    # Concatenar y promediar por articulación
    big = pd.concat(dfs, axis=0, keys=range(len(dfs)))  # MultiIndex (seq, articulación)
    mean_df = big.groupby(level=1).mean()               # (K articulaciones, columnas de métricas)

    # Sacar índice a columna explícita
    mean_df = mean_df.copy()
    mean_df.insert(0, "Articulación", mean_df.index)
    mean_df.reset_index(drop=True, inplace=True)
    return mean_df

# Promedio global ORIGINAL
mean_orig = promedio_por_articulacion(group_original)
if mean_orig is not None:
    mean_orig_name = "MPJPE_promedio_por_articulacion_original.csv"
    out_mean_orig = os.path.join(CSV_DIR, mean_orig_name)
    mean_orig.to_csv(out_mean_orig, index=False)
    # Añadir a grupo_promedio para que aparezca al final del .odt
    group_promedio[mean_orig_name] = out_mean_orig

# Promedio global CENTERED_HIP
mean_cent = promedio_por_articulacion(group_centered)
if mean_cent is not None:
    mean_cent_name = "MPJPE_promedio_por_articulacion_centered_hip.csv"
    out_mean_cent = os.path.join(CSV_DIR, mean_cent_name)
    mean_cent.to_csv(out_mean_cent, index=False)
    group_promedio[mean_cent_name] = out_mean_cent

# 6) Añadir tablas de promedios al final

if group_promedio:
    doc.text.addElement(P(stylename=section_style, text="Tablas de PROMEDIO global (MPJPE por articulación)"))
    for name in sorted(group_promedio.keys()):
        path = group_promedio[name]
        df = read_csv_for_table(path)
        add_df_table(doc, df, name)

# 7) Guardar y descargar
output_path = "tablas_ordenadas_con_promedios.odt"
doc.save(output_path)
files.download(output_path)
print("Documento generado:", output_path)



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.0/717.0 kB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
CSV originales encontrados : 20
CSV centrados encontrados  : 20
CSV de promedio ya existentes: 0


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Documento generado: tablas_ordenadas_con_promedios.odt


In [14]:
# PCK por frame y promedio por secuencia (centrado en Cadera_D=idx 6) y
# gráfica única por secuencia con curvas superpuestas de todos los MODELS.
import json, os, numpy as np, pandas as pd
import matplotlib.pyplot as plt

# === Config ===
JSON_PATH   = "/content/dataset_3models_norm_center_scale.ipynb_out.json"
PERSON_ID   = 1
MODELS      = ["mp", "vibe"]                 # claves de modelos en tu JSON
THRESHOLDS  = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.8, 1.0]  # misma unidad que k3d (ej. metros)
HIP_IDX     = 6                               # "Cadera_D"
OUTDIR      = "pck_frame_results_centered"
FIGDIR      = os.path.join(OUTDIR, "figs")
os.makedirs(FIGDIR, exist_ok=True)

# === Utilidades ===
def k3d_to_K3(mat):
    a = np.asarray(mat, dtype=float)
    if a.ndim != 2 or a.shape[0] < 3: raise ValueError(f"Esperado (3,K), recibido {a.shape}")
    return a[:3, :].T  # (K,3)

def find_person(persons_list, pid):
    cand = None
    for p in persons_list:
        if isinstance(p, dict) and p.get("id") == pid: cand = p; break
    if cand is None and persons_list: cand = persons_list[0]
    return cand.get("models", {}) if isinstance(cand, dict) else {}

def pck_frame(dists, taus):
    valid = np.isfinite(dists); tot = int(np.sum(valid))
    if tot == 0: return {tau: (0.0, 0) for tau in taus}
    vals = dists[valid]
    return {tau: (100.0 * float(np.mean(vals <= float(tau))), tot) for tau in taus}

def auc_trapz(x, y_percent):
    x = np.asarray(x, dtype=float); y = np.asarray(y_percent, dtype=float)/100.0
    if x.size < 2 or x[-1] == x[0]: return np.nan
    order = np.argsort(x); x, y = x[order], y[order]
    return float(np.trapz(y, x)) / (x[-1] - x[0])

# === Carga JSON ===
with open(JSON_PATH, "r") as f:
    data = json.load(f)
sequences = data.get("sequences", [])
if isinstance(sequences, dict): sequences = list(sequences.values())

rows_frame, rows_seq = [], []

# === Recorrido principal ===
for seq in sequences:
    sid = seq.get("video_id") or seq.get("sequence_id") or "unknown_seq"
    frames = seq.get("frames", [])
    # acumuladores por modelo para construir curvas y AUC superpuestos
    seq_acc = {m: {tau: [] for tau in THRESHOLDS} for m in MODELS}

    for fr in frames:
        models_dict = find_person(fr.get("persons", []), PERSON_ID)
        if "gt" not in models_dict or "k3d" not in models_dict["gt"]: continue
        try: gt_K3 = k3d_to_K3(models_dict["gt"]["k3d"])
        except: continue

        for m in MODELS:
            if m not in models_dict or "k3d" not in models_dict[m]: continue
            try: pr_K3 = k3d_to_K3(models_dict[m]["k3d"])
            except: continue

            K = min(pr_K3.shape[0], gt_K3.shape[0])
            pr, gt = pr_K3[:K].copy(), gt_K3[:K].copy()
            if HIP_IDX >= K: continue
            hip_pr, hip_gt = pr[HIP_IDX], gt[HIP_IDX]
            if not (np.all(np.isfinite(hip_pr)) and np.all(np.isfinite(hip_gt))): continue
            pr -= hip_pr; gt -= hip_gt

            dists = np.linalg.norm(pr - gt, axis=1)  # (K,)
            per_tau = pck_frame(dists, THRESHOLDS)
            for tau, (pckp, totv) in per_tau.items():
                rows_frame.append({
                    "sequence_id": sid, "frame": fr.get("f", None),
                    "model": m, "tau": float(tau),
                    "pck_percent": round(pckp, 6), "valid_keypoints": int(totv)
                })
                seq_acc[m][tau].append(pckp)

    # === Curva superpuesta por secuencia (todos los modelos) + AUC en leyenda ===
    plt.figure()
    plotted = False
    for m in MODELS:
        ys = [float(np.mean(seq_acc[m][tau])) if seq_acc[m][tau] else 0.0 for tau in THRESHOLDS]
        if any(v != 0.0 for v in ys):
            auc_norm = auc_trapz(THRESHOLDS, ys)
            plt.plot(THRESHOLDS, ys, marker="o", label=f"{m} | AUC={auc_norm:.3f}")
            for tau, yv in zip(THRESHOLDS, ys):
                rows_seq.append({"sequence_id": sid, "model": m, "tau": float(tau),
                                 "pck_percent_mean": round(yv, 6),
                                 "frames_used": len(seq_acc[m][tau])})
            plotted = True
    if plotted:
        plt.xlabel("Umbral (misma unidad que k3d)")
        plt.ylabel("PCK (%)")
        plt.title(f"{sid} – Curvas PCK (centrado en Cadera_D)")
        plt.grid(True); plt.legend(); plt.tight_layout()
        plt.savefig(os.path.join(FIGDIR, f"{sid}__pck_curves_centered.png"))
    plt.close()

# === Guardado de CSV ===
pd.DataFrame(rows_frame).sort_values(["sequence_id","model","frame","tau"]).to_csv(
    os.path.join(OUTDIR, "pck_per_frame_centered.csv"), index=False
)
pd.DataFrame(rows_seq).sort_values(["sequence_id","model","tau"]).to_csv(
    os.path.join(OUTDIR, "pck_per_sequence_mean_centered.csv"), index=False
)

print(f"OK → {OUTDIR}/pck_per_frame_centered.csv, pck_per_sequence_mean_centered.csv y figuras en {FIGDIR}/ (curvas superpuestas)")

# Zip y descarga la carpeta pck_results (Colab).
import shutil
zip_path = shutil.make_archive('pck_frame_results_centered', 'zip', 'pck_frame_results_centered')
from google.colab import files
files.download(zip_path)

/tmp/ipython-input-2681309860.py:39: DeprecationWarning:

`trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.



OK → pck_frame_results_centered/pck_per_frame_centered.csv, pck_per_sequence_mean_centered.csv y figuras en pck_frame_results_centered/figs/ (curvas superpuestas)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>